In [0]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [0]:
olympics=pd.read_csv('/content/drive/My Drive/DataMiningProjecct/athlete_events.csv')

In [40]:
#Printing column wise missing values
print(olympics.isnull().sum())

ID             0
Name           0
Sex            0
Age         9474
Height     60171
Weight     62875
Team           0
NOC            0
Games          0
Year           0
Season         0
City           0
Sport          0
Event          0
Medal     231333
dtype: int64


In [0]:
#replace these missing values in medals by 'Did not win' or 'DNW' for the atheletes who did not win any medals
olympics['Medal'].fillna('DNW', inplace = True)

In [0]:
#reading NOC dataset
noc_country = pd.read_csv('/content/drive/My Drive/DataMiningProjecct/noc_regions.csv')
noc_country.drop('notes', axis = 1 , inplace = True)
noc_country.rename(columns = {'region':'Country'}, inplace = True)

In [0]:
#joining both datasets with NOC as primary key
olympics_merge = olympics.merge(noc_country,
                                left_on = 'NOC',
                                right_on = 'NOC',
                                how = 'left')

In [44]:
# Do we have NOCs that didnt have a matching country in the master?
print(olympics_merge.loc[olympics_merge['Country'].isnull(),['NOC', 'Team']].drop_duplicates())

        NOC                      Team
578     SGP                 Singapore
6267    ROT  Refugee Olympic Athletes
44376   SGP              June Climene
61080   UNK                   Unknown
64674   TUV                    Tuvalu
80986   SGP                   Rika II
108582  SGP               Singapore-2
235895  SGP               Singapore-1


In [0]:
# Replace missing Teams by the values 1. SGP - Singapore
                                    # 2. ROT - Refugee Olympic Athletes
                                    # 3. UNK - Unknown
                                    # 4. TUV - Tuvalu
#olympics_merge.loc[olympics_merge['Country'].isnull(), ['Country']] = olympics_merge['Team']

olympics_merge['Country'] = np.where(olympics_merge['NOC']=='SGP', 'Singapore', olympics_merge['Country'])
olympics_merge['Country'] = np.where(olympics_merge['NOC']=='ROT', 'Refugee Olympic Athletes', olympics_merge['Country'])
olympics_merge['Country'] = np.where(olympics_merge['NOC']=='UNK', 'Unknown', olympics_merge['Country'])
olympics_merge['Country'] = np.where(olympics_merge['NOC']=='TUV', 'Tuvalu', olympics_merge['Country'])

In [0]:
# Drop Team column and rename Country column to team column
olympics_merge.drop('Team', axis = 1, inplace = True)
olympics_merge.rename(columns = {'Country': 'Team'}, inplace = True)

In [47]:
#Checking again for mapping of NOC to team we find that each is mapped to a single value.
print(olympics_merge.loc[olympics_merge['Team'].isnull(),['NOC', 'Team']].drop_duplicates())

Empty DataFrame
Columns: [NOC, Team]
Index: []


In [48]:
#checking Null Values
print(olympics_merge.isnull().sum())

ID            0
Name          0
Sex           0
Age        9474
Height    60171
Weight    62875
NOC           0
Games         0
Year          0
Season        0
City          0
Sport         0
Event         0
Medal         0
Team          0
dtype: int64


In [49]:
# Lets take data from 1961 onwards only and for summer olympics only
olympics_complete_subset = olympics_merge.loc[(olympics_merge['Year'] > 1960) & (olympics_merge['Season'] == "Summer"), :]
print(olympics_complete_subset.head())

    ID                Name Sex  ...                         Event  Medal     Team
0    1           A Dijiang   M  ...   Basketball Men's Basketball    DNW    China
1    2            A Lamusi   M  ...  Judo Men's Extra-Lightweight    DNW    China
31  12   Jyri Tapani Aalto   M  ...       Badminton Men's Singles    DNW  Finland
32  13  Minna Maarit Aalto   F  ...    Sailing Women's Windsurfer    DNW  Finland
33  13  Minna Maarit Aalto   F  ...    Sailing Women's Windsurfer    DNW  Finland

[5 rows x 15 columns]


In [50]:
# We see the row indices are not contigent anymore due to removal of rows. So,Reset row indices
olympics_complete_subset = olympics_complete_subset.reset_index()
print(olympics_complete_subset.head())

   index  ID                Name  ...                         Event  Medal     Team
0      0   1           A Dijiang  ...   Basketball Men's Basketball    DNW    China
1      1   2            A Lamusi  ...  Judo Men's Extra-Lightweight    DNW    China
2     31  12   Jyri Tapani Aalto  ...       Badminton Men's Singles    DNW  Finland
3     32  13  Minna Maarit Aalto  ...    Sailing Women's Windsurfer    DNW  Finland
4     33  13  Minna Maarit Aalto  ...    Sailing Women's Windsurfer    DNW  Finland

[5 rows x 16 columns]


In [51]:
#checking Null Values
print(olympics_complete_subset.isnull().sum())
#Null values reduced considerably by using the method of discarding tuples

index        0
ID           0
Name         0
Sex          0
Age        827
Height    7030
Weight    7294
NOC          0
Games        0
Year         0
Season       0
City         0
Sport        0
Event        0
Medal        0
Team         0
dtype: int64


In [52]:
print(olympics_complete_subset)

         index      ID  ... Medal       Team
0            0       1  ...   DNW      China
1            1       2  ...   DNW      China
2           31      12  ...   DNW    Finland
3           32      13  ...   DNW    Finland
4           33      13  ...   DNW    Finland
...        ...     ...  ...   ...        ...
158143  271106  135565  ...   DNW  Argentina
158144  271107  135566  ...   DNW        USA
158145  271108  135567  ...   DNW     Russia
158146  271109  135567  ...   DNW     Russia
158147  271110  135568  ...   DNW    Belarus

[158148 rows x 16 columns]


In [53]:
#Extracting unique events in a new list
listunique=olympics_complete_subset.Event.unique()
print(listunique)
print(len(listunique))

["Basketball Men's Basketball" "Judo Men's Extra-Lightweight"
 "Badminton Men's Singles" "Sailing Women's Windsurfer"
 "Athletics Men's Shot Put" "Handball Women's Handball"
 "Weightlifting Women's Super-Heavyweight"
 "Wrestling Men's Light-Heavyweight, Greco-Roman"
 "Water Polo Men's Water Polo" 'Sailing Mixed Three Person Keelboat'
 "Hockey Women's Hockey" "Rowing Men's Lightweight Double Sculls"
 "Sailing Men's Two Person Dinghy" "Athletics Men's 1,500 metres"
 "Swimming Men's 100 metres Butterfly"
 "Swimming Men's 200 metres Butterfly"
 "Swimming Men's 4 x 100 metres Medley Relay" "Football Women's Football"
 'Equestrianism Mixed Jumping, Individual'
 "Boxing Men's Light-Welterweight" "Athletics Women's Javelin Throw"
 "Wrestling Men's Heavyweight, Freestyle"
 "Gymnastics Men's Individual All-Around"
 "Gymnastics Men's Floor Exercise" "Gymnastics Men's Parallel Bars"
 "Gymnastics Men's Horizontal Bar" "Gymnastics Men's Rings"
 "Gymnastics Men's Pommelled Horse" "Athletics Men's Pol

In [54]:
tempagemean=[]
tempweightmean=[]
tempheightmean=[]

#looping to the event list and creating new lists for means of age,weight and height for unique events.
for i in listunique:
    temporary=olympics_complete_subset.loc[(olympics_complete_subset['Event'] == i) & (olympics_complete_subset['Age'] !='NaN' ), :]
    tempagemean.append(temporary.Age.mean(axis = 0, skipna = True))
    tempweightmean.append(temporary.Weight.mean(axis=0,skipna = True))
    tempheightmean.append(temporary.Height.mean(axis = 0, skipna = True))

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [55]:
#looping for each event(since there are 372 events)
for i in range(0,372):
    #extracting rows with a particular event into another dataframe
    df1 = olympics_complete_subset[olympics_complete_subset['Event'].str.contains(listunique[i]) ]
    #filling null values for age in the new dataframe to the mean age for that event
    df1.Age.fillna(tempagemean[i],inplace=True)
    # filling null values for weight in the new dataframe to the mean weight for that event
    df1.Weight.fillna(tempweightmean[i], inplace=True)
    # filling null values for height in the new dataframe to the mean height for that event
    df1.Height.fillna(tempheightmean[i], inplace=True)
    #Dropping columns for that particular event in the original dataframe
    olympics_complete_subset.drop(olympics_complete_subset[olympics_complete_subset['Event'].str.contains(listunique[i])].index, inplace = True)
    #concatinating the altered rows in new dataframe to the old dataframe
    olympics_complete_subset = pd.concat([olympics_complete_subset, df1], axis=0)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [56]:
#Print statements to check for null values
print(olympics_complete_subset.head())
print(olympics_complete_subset.isnull().sum())
print(olympics_complete_subset)

     index   ID  ... Medal   Team
0        0    1  ...   DNW  China
135    264  136  ...   DNW  Italy
187    346  192  ...   DNW  Egypt
191    359  199  ...   DNW  Egypt
192    360  199  ...   DNW  Egypt

[5 rows x 16 columns]
index     0
ID        0
Name      0
Sex       0
Age       0
Height    0
Weight    0
NOC       0
Games     0
Year      0
Season    0
City      0
Sport     0
Event     0
Medal     0
Team      0
dtype: int64
         index      ID  ...   Medal     Team
0            0       1  ...     DNW    China
135        264     136  ...     DNW    Italy
187        346     192  ...     DNW    Egypt
191        359     199  ...     DNW    Egypt
192        360     199  ...     DNW    Egypt
...        ...     ...  ...     ...      ...
147814  253711  127043  ...  Bronze  Germany
148315  254866  127585  ...  Silver   Russia
150488  258804  129570  ...    Gold  Germany
150491  258807  129570  ...  Bronze  Germany
157148  269368  134783  ...    Gold    China

[158148 rows x 16 columns]


In [0]:
#giving numeric value to gender column 1 for male and 0 for female
olympics_complete_subset['Sex'] = np.where(olympics_complete_subset['Sex']=='M', 1 , 0)

In [58]:
#Creating a column that captures whether or not a medal was won! It would be 1 if Medal column says Gold, Silver or Bronze and 0 otherwise.
#Converting Medal column to numeric form
olympics_complete_subset['Medal_Won'] = np.where(olympics_complete_subset.loc[:,'Medal'] == 'DNW', 0 , olympics_complete_subset['Medal'])
olympics_complete_subset['Medal_Won'] = np.where(olympics_complete_subset.loc[:,'Medal_Won'] == 'Bronze', 1 , olympics_complete_subset['Medal_Won'])
olympics_complete_subset['Medal_Won'] = np.where(olympics_complete_subset.loc[:,'Medal_Won'] == 'Silver', 2 , olympics_complete_subset['Medal_Won'])
olympics_complete_subset['Medal_Won'] = np.where(olympics_complete_subset.loc[:,'Medal_Won'] == 'Gold', 3 , olympics_complete_subset['Medal_Won'])
print(olympics_complete_subset['Medal_Won'].unique())

[0 3 1 2]


In [0]:
#Dropping unnecessary columns from dataFrame
olympics_complete_subset.drop(['index'] , axis=1, inplace=True)
olympics_complete_subset.drop(['Name'] , axis=1, inplace=True)
olympics_complete_subset.drop(['NOC'] , axis=1, inplace=True)
olympics_complete_subset.drop(['Season'] , axis=1, inplace=True)
olympics_complete_subset.drop(['Games'] , axis=1, inplace=True)
olympics_complete_subset.drop(['City'] , axis=1, inplace=True)
olympics_complete_subset.drop(['Year'] , axis=1, inplace=True)
olympics_complete_subset.drop(['Sport'] , axis=1, inplace=True)
olympics_complete_subset.drop(['Medal'] , axis=1, inplace=True)
olympics_complete_subset.drop(['ID'] , axis=1, inplace=True)

In [0]:
# We see the row indices are not contigent anymore due to removal of rows. So,Reset row indices
olympics_complete_subset = olympics_complete_subset.reset_index()

#converting Team and Event to numerical form

In [0]:
#Creating object of preprocessing LabelEncoder
le = preprocessing.LabelEncoder()
#Creating a new column of encoded Team and Encoded Event in numerical form in original dataset
olympics_complete_subset['Team_encode'] = le.fit_transform(olympics_complete_subset['Team'])
olympics_complete_subset['Event_encode'] = le.fit_transform(olympics_complete_subset['Event'])

In [62]:
#storing Team names and corresponding  encoded numerical values into new csv file after sorting them according to Team name
TeamKeys=olympics_complete_subset[['Team', 'Team_encode']].copy()
TeamKeys.drop_duplicates(subset ="Team", inplace = True)
TeamKeys.sort_values("Team", axis = 0, ascending = True, inplace = True, na_position ='last')
TeamKeys.to_csv('/content/drive/My Drive/DataMiningProjecct/Preprocessed-datasets/keysToTeam.csv')
print(TeamKeys.head())

                 Team  Team_encode
24011     Afghanistan            0
3106          Albania            1
2272          Algeria            2
3154   American Samoa            3
2351          Andorra            4


In [63]:
#storing Event names and corresponding  encoded numerical values into new csv file after sorting them according to Event name
EventKeys=olympics_complete_subset[['Event' , 'Event_encode']].copy()
EventKeys.drop_duplicates(subset ="Event", inplace = True)
EventKeys.sort_values("Event", axis = 0, ascending = True, inplace = True, na_position ='last')
EventKeys.to_csv('/content/drive/My Drive/DataMiningProjecct/Preprocessed-datasets/keysToEvent.csv')
print(EventKeys.head())

                               Event  Event_encode
117345      Archery Men's Individual             0
118080            Archery Men's Team             1
62659     Archery Women's Individual             2
130135          Archery Women's Team             3
10085   Athletics Men's 1,500 metres             4


In [64]:
#Creating a new dataframe to store keys to Sex and store it in new csv file
SexKeys = pd.DataFrame( {'Sex': ['Male','Female'], 'Sex_encode': [1,0] }, index =[1,2] )
SexKeys.to_csv('/content/drive/My Drive/DataMiningProjecct/Preprocessed-datasets/keysToSex.csv')
print(SexKeys)

      Sex  Sex_encode
1    Male           1
2  Female           0


In [0]:
'''after mapping the key value pairs, dropping the team and Event columns from final dataset and changing
names of Team_encode to Team and Event_encode to Event in the original dataframe'''
olympics_complete_subset.drop(['Team'] , axis=1, inplace=True)
olympics_complete_subset.drop(['Event'] , axis=1, inplace=True)
olympics_complete_subset.rename(columns = {'Team_encode': 'Team'}, inplace = True)
olympics_complete_subset.rename(columns = {'Event_encode': 'Event'}, inplace = True)

In [66]:
olympics_complete_subset.drop(['index'] , axis=1, inplace=True)
#Converting the final dataset into a new csv file
olympics_complete_subset.to_csv('/content/drive/My Drive/DataMiningProjecct/Preprocessed-datasets/olympics_final_final.csv');
print(olympics_complete_subset.head())

   Sex   Age  Height  Weight Medal_Won  Team  Event
0    1  24.0   180.0    80.0         0    39     61
1    1  29.0   195.0    85.0         0    87     61
2    1  25.0   189.0    85.0         0    55     61
3    1  23.0   178.0    67.0         0    55     61
4    1  27.0   178.0    67.0         0    55     61
